In [3]:
import ccxt
import pandas as pd
import time

# Initialize the exchange
exchange = ccxt.binance()

# Load all markets with a retry mechanism
max_retries = 3
for attempt in range(max_retries):
    try:
        markets = exchange.load_markets()
        break
    except Exception as e:
        print(f"Attempt {attempt+1} failed: {e}")
        if attempt < max_retries - 1:
            time.sleep(2)  # wait before retrying
        else:
            raise

# Filter relevant pairs
usdt_pairs = [symbol for symbol in markets if symbol.endswith('/USDT')]
eth_pairs = [symbol for symbol in markets if symbol.endswith('/ETH')]
eth_usdt_pair = 'ETH/USDT'

# Ensure ETH/USDT pair is available
if eth_usdt_pair not in markets:
    raise ValueError("ETH/USDT pair not available on Binance")

# Create DataFrame to store market data
columns = ['base', 'quote1', 'quote2', 'symbol1', 'symbol2', 'symbol3', 'bid1', 'ask1', 'bid2', 'ask2', 'bid3', 'ask3']
tri_arb_data = []

# Function to fetch ticker data with retry mechanism
def fetch_ticker_data(symbol):
    for attempt in range(max_retries):
        try:
            ticker = exchange.fetch_ticker(symbol)
            return ticker['bid'], ticker['ask']
        except Exception as e:
            print(f"Error fetching data for {symbol} on attempt {attempt+1}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)  # wait before retrying
            else:
                raise

# Find triangular arbitrage opportunities
for usdt_pair in usdt_pairs:
    base_currency = usdt_pair.split('/')[0]
    eth_pair = f"{base_currency}/ETH"
    
    if eth_pair in eth_pairs:
        try:
            bid1, ask1 = fetch_ticker_data(usdt_pair)
            bid2, ask2 = fetch_ticker_data(eth_pair)
            bid3, ask3 = fetch_ticker_data(eth_usdt_pair)
            
            tri_arb_data.append([
                base_currency, 'USDT', 'ETH', usdt_pair, eth_pair, eth_usdt_pair,
                bid1, ask1, bid2, ask2, bid3, ask3
            ])
        except Exception as e:
            print(f"Error fetching data for {base_currency}: {e}")

# Create DataFrame
df = pd.DataFrame(tri_arb_data, columns=columns)

# Print the DataFrame
print(df)

# Optionally, save the DataFrame to a CSV file
df.to_csv('binance_triangular_arbitrage_pairs.csv', index=False)


      base quote1 quote2     symbol1    symbol2   symbol3        bid1  \
0      BNB   USDT    ETH    BNB/USDT    BNB/ETH  ETH/USDT   628.50000   
1      BCC   USDT    ETH    BCC/USDT    BCC/ETH  ETH/USDT         NaN   
2      NEO   USDT    ETH    NEO/USDT    NEO/ETH  ETH/USDT    14.63000   
3      LTC   USDT    ETH    LTC/USDT    LTC/ETH  ETH/USDT    82.06000   
4     QTUM   USDT    ETH   QTUM/USDT   QTUM/ETH  ETH/USDT     3.44400   
..     ...    ...    ...         ...        ...       ...         ...   
158    SNT   USDT    ETH    SNT/USDT    SNT/ETH  ETH/USDT     0.03490   
159  WBETH   USDT    ETH  WBETH/USDT  WBETH/ETH  ETH/USDT  3916.68000   
160  CYBER   USDT    ETH  CYBER/USDT  CYBER/ETH  ETH/USDT     8.91200   
161    ARK   USDT    ETH    ARK/USDT    ARK/ETH  ETH/USDT     0.76600   
162   MEME   USDT    ETH   MEME/USDT   MEME/ETH  ETH/USDT     0.02882   

           ask1      bid2      ask2     bid3     ask3  
0     628.60000  0.166800  0.166900  3766.61  3766.62  
1          

In [4]:
df.head()

,base,quote1,quote2,symbol1,symbol2,symbol3,bid1,ask1,bid2,ask2,bid3,ask3
0,BNB,USDT,ETH,BNB/USDT,BNB/ETH,ETH/USDT,628.500,628.600,0.166800,0.166900,3766.61,3766.62
1,BCC,USDT,ETH,BCC/USDT,BCC/ETH,ETH/USDT,NaN,NaN,NaN,NaN,3766.76,3766.77
2,NEO,USDT,ETH,NEO/USDT,NEO/ETH,ETH/USDT,14.630,14.640,NaN,NaN,3766.76,3766.77
3,LTC,USDT,ETH,LTC/USDT,LTC/ETH,ETH/USDT,82.060,82.070,0.021780,0.021790,3766.76,3766.77
4,QTUM,USDT,ETH,QTUM/USDT,QTUM/ETH,ETH/USDT,3.444,3.445,0.000913,0.000915,3766.76,3766.77


In [7]:
# Calculate potential profit
df['profit'] = (
    (1 / df['ask1']) *  # Start with USDT, buy BASE at ask1 price
    (1 / df['ask2']) *  # Then buy ETH with BASE at ask2 price
    df['bid3'] -        # Finally sell ETH for USDT at bid3 price
    1                   # Subtract initial 1 USDT
) * 100  # Convert to percentage

# Filter for positive profit opportunities
df2 = df[df['profit'] > 0]

# Format the profit column to 2 decimal places
df2['profit'] = df2['profit'].map('{:.2f}'.format)


# Optionally, save the DataFrame to a CSV file
# df2.to_csv('binance_triangular_arbitrage_opportunities.csv', index=False)
df2.head()

/tmp/ipykernel_171362/3201564650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['profit'] = df2['profit'].map('{:.2f}'.format)


,base,quote1,quote2,symbol1,symbol2,symbol3,bid1,ask1,bid2,ask2,bid3,ask3,profit
0,BNB,USDT,ETH,BNB/USDT,BNB/ETH,ETH/USDT,628.5000,628.6000,0.166800,0.166900,3766.61,3766.62,3490.21
3,LTC,USDT,ETH,LTC/USDT,LTC/ETH,ETH/USDT,82.0600,82.0700,0.021780,0.021790,3766.76,3766.77,210532.94
4,QTUM,USDT,ETH,QTUM/USDT,QTUM/ETH,ETH/USDT,3.4440,3.4450,0.000913,0.000915,3766.76,3766.77,119497072.59
5,ADA,USDT,ETH,ADA/USDT,ADA/ETH,ETH/USDT,0.4560,0.4561,0.000121,0.000121,3766.76,3766.77,6819675782.72
6,XRP,USDT,ETH,XRP/USDT,XRP/ETH,ETH/USDT,0.5188,0.5189,0.000138,0.000138,3766.76,3766.77,5267870054.04


# Second try

In [8]:
import ccxt
import pandas as pd
import time

# Initialize the exchange
exchange = ccxt.binance()

# Load all markets with a retry mechanism
max_retries = 3
for attempt in range(max_retries):
    try:
        markets = exchange.load_markets()
        break
    except Exception as e:
        print(f"Attempt {attempt+1} failed: {e}")
        if attempt < max_retries - 1:
            time.sleep(2)  # wait before retrying
        else:
            raise

# Filter relevant pairs
usdt_pairs = [symbol for symbol in markets if symbol.endswith('/USDT')]
eth_pairs = [symbol for symbol in markets if symbol.endswith('/ETH')]
eth_usdt_pair = 'ETH/USDT'

# Ensure ETH/USDT pair is available
if eth_usdt_pair not in markets:
    raise ValueError("ETH/USDT pair not available on Binance")

# Create DataFrame to store market data
columns = ['base', 'quote1', 'quote2', 'symbol1', 'symbol2', 'symbol3', 'bid1', 'ask1', 'bid2', 'ask2', 'bid3', 'ask3']
tri_arb_data = []

# Function to fetch ticker data with retry mechanism
def fetch_ticker_data(symbol):
    for attempt in range(max_retries):
        try:
            ticker = exchange.fetch_ticker(symbol)
            return ticker['bid'], ticker['ask']
        except Exception as e:
            print(f"Error fetching data for {symbol} on attempt {attempt+1}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)  # wait before retrying
            else:
                raise

# Find triangular arbitrage opportunities
for usdt_pair in usdt_pairs:
    base_currency = usdt_pair.split('/')[0]
    eth_pair = f"{base_currency}/ETH"
    
    if eth_pair in eth_pairs:
        try:
            bid1, ask1 = fetch_ticker_data(usdt_pair)
            bid2, ask2 = fetch_ticker_data(eth_pair)
            bid3, ask3 = fetch_ticker_data(eth_usdt_pair)
            
            tri_arb_data.append([
                base_currency, 'USDT', 'ETH', usdt_pair, eth_pair, eth_usdt_pair,
                bid1, ask1, bid2, ask2, bid3, ask3
            ])
        except Exception as e:
            print(f"Error fetching data for {base_currency}: {e}")

# Create DataFrame
df = pd.DataFrame(tri_arb_data, columns=columns)

# Calculate potential profit
# Step-by-step arbitrage calculation
# 1. Start with 1 USDT
# 2. Buy BASE with USDT at ask1 price
# 3. Sell BASE for ETH at bid2 price
# 4. Sell ETH for USDT at bid3 price

df['profit'] = (
    (1 / df['ask1']) *  # Step 2: Buy BASE with USDT at ask1 price
    df['bid2'] *        # Step 3: Sell BASE for ETH at bid2 price
    df['bid3'] -        # Step 4: Sell ETH for USDT at bid3 price
    1                   # Subtract initial 1 USDT
) * 100  # Convert to percentage

# Filter for positive profit opportunities
df2 = df[df['profit'] > 0]

# Format the profit column to 2 decimal places
df2['profit'] = df2['profit'].map('{:.2f}'.format)

# Print the DataFrame with opportunities
# print(df2)

# Optionally, save the DataFrame to a CSV file
# df2.to_csv('binance_triangular_arbitrage_opportunities.csv', index=False)
df2.head()

/tmp/ipykernel_171362/2728043202.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['profit'] = df2['profit'].map('{:.2f}'.format)


,base,quote1,quote2,symbol1,symbol2,symbol3,bid1,ask1,bid2,ask2,bid3,ask3,profit
12,TRX,USDT,ETH,TRX/USDT,TRX/ETH,ETH/USDT,0.11344,0.11345,0.000030,0.000030,3771.98,3771.99,0.01
90,UNI,USDT,ETH,UNI/USDT,UNI/ETH,ETH/USDT,9.49500,9.49600,0.002519,0.002521,3770.25,3770.26,0.01
93,NEAR,USDT,ETH,NEAR/USDT,NEAR/ETH,ETH/USDT,7.06000,7.06100,0.001873,0.001875,3770.00,3770.01,0.00
159,WBETH,USDT,ETH,WBETH/USDT,WBETH/ETH,ETH/USDT,3919.80000,3919.81000,1.039900,1.040000,3769.61,3769.62,0.01


# Third Try

In [9]:
import ccxt
import pandas as pd
import time

# Initialize the exchange
exchange = ccxt.binance()

# Load all markets with a retry mechanism
max_retries = 3
for attempt in range(max_retries):
    try:
        markets = exchange.load_markets()
        break
    except Exception as e:
        print(f"Attempt {attempt+1} failed: {e}")
        if attempt < max_retries - 1:
            time.sleep(2)  # wait before retrying
        else:
            raise

# Filter relevant pairs
usdt_pairs = [symbol for symbol in markets if symbol.endswith('/USDT')]
eth_pairs = [symbol for symbol in markets if symbol.endswith('/ETH')]
eth_usdt_pair = 'ETH/USDT'

# Ensure ETH/USDT pair is available
if eth_usdt_pair not in markets:
    raise ValueError("ETH/USDT pair not available on Binance")

# Create DataFrame to store market data
columns = ['base', 'quote1', 'quote2', 'symbol1', 'symbol2', 'symbol3', 'bid1', 'ask1', 'bid2', 'ask2', 'bid3', 'ask3']
tri_arb_data = []

# Function to fetch ticker data with retry mechanism
def fetch_ticker_data(symbol):
    for attempt in range(max_retries):
        try:
            ticker = exchange.fetch_ticker(symbol)
            return ticker['bid'], ticker['ask']
        except Exception as e:
            print(f"Error fetching data for {symbol} on attempt {attempt+1}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)  # wait before retrying
            else:
                raise

# Find triangular arbitrage opportunities
for usdt_pair in usdt_pairs:
    base_currency = usdt_pair.split('/')[0]
    eth_pair = f"{base_currency}/ETH"
    
    if eth_pair in eth_pairs:
        try:
            bid1, ask1 = fetch_ticker_data(usdt_pair)
            bid2, ask2 = fetch_ticker_data(eth_pair)
            bid3, ask3 = fetch_ticker_data(eth_usdt_pair)
            
            tri_arb_data.append([
                base_currency, 'USDT', 'ETH', usdt_pair, eth_pair, eth_usdt_pair,
                bid1, ask1, bid2, ask2, bid3, ask3
            ])
        except Exception as e:
            print(f"Error fetching data for {base_currency}: {e}")

# Create DataFrame
df = pd.DataFrame(tri_arb_data, columns=columns)

# Calculate potential profit
df['profit'] = (
    (1 / df['ask1']) *  # Step 2: Buy BASE with USDT at ask1 price
    df['bid2'] *        # Step 3: Sell BASE for ETH at bid2 price
    df['bid3'] -        # Step 4: Sell ETH for USDT at bid3 price
    1                   # Subtract initial 1 USDT
) * 100  # Convert to percentage

# Filter for positive profit opportunities
df2 = df[df['profit'] > 0]

# Format the profit column to 2 decimal places
df2['profit'] = df2['profit'].map('{:.2f}'.format)


# Optionally, save the DataFrame to a CSV file
# df2.to_csv('binance_triangular_arbitrage_opportunities.csv', index=False)
df2.head()


/tmp/ipykernel_171362/692088646.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['profit'] = df2['profit'].map('{:.2f}'.format)


,base,quote1,quote2,symbol1,symbol2,symbol3,bid1,ask1,bid2,ask2,bid3,ask3,profit
156,WBTC,USDT,ETH,WBTC/USDT,WBTC/ETH,ETH/USDT,68930.93,68954.87,18.29,18.3,3770.59,3770.6,0.01


In [12]:
# Calculate final profit with 100 USDT initial investment and 0.10% fee per trade
initial_investment = 5000
fee_rate = 0.001

def calculate_final_profit(row):
    # Step 1: Start with 100 USDT
    usdt_amount = initial_investment
    
    # Step 2: Buy BASE with USDT at ask1 price, considering the fee
    base_amount = (usdt_amount / row['ask1']) * (1 - fee_rate)
    
    # Step 3: Sell BASE for ETH at bid2 price, considering the fee
    eth_amount = (base_amount * row['bid2']) * (1 - fee_rate)
    
    # Step 4: Sell ETH for USDT at bid3 price, considering the fee
    final_usdt_amount = (eth_amount * row['bid3']) * (1 - fee_rate)
    
    # Calculate final profit
    final_profit = final_usdt_amount - initial_investment
    return final_profit

df2['final_profit'] = df2.apply(calculate_final_profit, axis=1)

# Format the final profit column to 2 decimal places
df2['final_profit'] = df2['final_profit'].map('{:.2f}'.format)

# Print the DataFrame with opportunities
df2.head()


/tmp/ipykernel_171362/1096656598.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['final_profit'] = df2.apply(calculate_final_profit, axis=1)
/tmp/ipykernel_171362/1096656598.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['final_profit'] = df2['final_profit'].map('{:.2f}'.format)


,base,quote1,quote2,symbol1,symbol2,symbol3,bid1,ask1,bid2,ask2,bid3,ask3,profit,final_profit
156,WBTC,USDT,ETH,WBTC/USDT,WBTC/ETH,ETH/USDT,68930.93,68954.87,18.29,18.3,3770.59,3770.6,0.01,-14.32
